# Import Libraries

In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read Datasets

In [2]:
!unzip "/content/drive/MyDrive/Assignment.zip"

Archive:  /content/drive/MyDrive/Assignment.zip
  inflating: Problem Statement.pdf   
   creating: data/
  inflating: data/customers.csv      
  inflating: data/pings.csv          
  inflating: data/.DS_Store          
  inflating: data/test.csv           
  inflating: data/Data Description.pdf  


In [3]:
df = pd.read_csv('/content/data/pings.csv')
df2 = pd.read_csv('/content/data/test.csv')
df3 = pd.read_csv('/content/data/customers.csv')

In [4]:
df.head()

,id,timestamp
0,899313,1496278800
1,373017,1496278800
2,798984,1496278800
3,245966,1496278800
4,689783,1496278800


# Transformations

In [5]:
# feature engineering - year, month,daym hour, minute from timestamp
df['date'] = pd.to_datetime(df['timestamp'], unit='s')

In [6]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df['min'] = df['date'].dt.minute

In [7]:
df.head()

,id,timestamp,date,year,month,day,hour,min
0,899313,1496278800,2017-06-01 01:00:00,2017,6,1,1,0
1,373017,1496278800,2017-06-01 01:00:00,2017,6,1,1,0
2,798984,1496278800,2017-06-01 01:00:00,2017,6,1,1,0
3,245966,1496278800,2017-06-01 01:00:00,2017,6,1,1,0
4,689783,1496278800,2017-06-01 01:00:00,2017,6,1,1,0


In [8]:
# feature selection
for i in df.columns:
    print(i,df[i].nunique())

id 2480
timestamp 121415
date 121415
year 1
month 1
day 22
hour 24
min 60


In [9]:
df = df[['id','day', 'hour', 'min']]

In [10]:
# Drop duplicate rows (if any)
df = df.drop_duplicates()

In [11]:
# sort rows by id, date
df = df.sort_values(by=list(df.columns))

In [12]:
# order index of rows
df = df.reset_index(drop=True)

In [13]:
# get online hours column (target) 
df4 = df.groupby(by=['id','day','hour']).size().reset_index(name='counts')
df4.head()

,id,day,hour,counts
0,111556,1,1,51
1,111556,1,2,60
2,111556,1,3,23
3,111556,2,1,38
4,111556,2,2,60


In [14]:
df4['online_hours'] = df4['counts']/60
df4.head()

,id,day,hour,counts,online_hours
0,111556,1,1,51,0.850000
1,111556,1,2,60,1.000000
2,111556,1,3,23,0.383333
3,111556,2,1,38,0.633333
4,111556,2,2,60,1.000000


In [15]:
df5 = df4.groupby(by=['id','day']).sum().reset_index()
df5.head()

,id,day,hour,counts,online_hours
0,111556,1,6,134,2.233333
1,111556,2,6,151,2.516667
2,111556,5,21,281,4.683333
3,111556,6,10,188,3.133333
4,111556,7,10,159,2.650000


In [16]:
# join with customers data
def prep(df):
    df = pd.merge(df,df3,on='id')
    df = df[['gender','age','number_of_kids','day','online_hours']]
    df = df.dropna()
    df = df.drop_duplicates()
    return df

In [17]:
train = prep(df5)
train.head()

,gender,age,number_of_kids,day,online_hours
0,FEMALE,49,4,1,2.233333
1,FEMALE,49,4,2,2.516667
2,FEMALE,49,4,5,4.683333
3,FEMALE,49,4,6,3.133333
4,FEMALE,49,4,7,2.650000


In [18]:
train.to_csv('train.csv',index=False)

In [19]:
df2['date'] = pd.to_datetime(df2['date'])
df2['day'] = df2['date'].dt.day

In [20]:
test = prep(df2)
test.head()

,gender,age,number_of_kids,day,online_hours
0,MALE,26,2,28,7
1,MALE,26,2,27,9
2,MALE,26,2,26,9
3,MALE,26,2,25,10
4,MALE,26,2,24,9


In [21]:
test.to_csv('test.csv',index=False)